Goal: Find the most active electricity firms in the australian market.
I need to be able to sort between sophisticated and non-sophisticated firms

Australia’s electricity generation landscape is dominated by several key players and power stations.

Major Electricity Generators:
	•	AGL Energy: As of recent data, AGL operates Australia’s largest electricity generation portfolio within the National Electricity Market and is a leading private investor in renewable energy.  ￼
	•	Origin Energy: Another significant player, Origin Energy, owns and operates substantial generation assets, including the Eraring Power Station in New South Wales, which is one of the country’s largest.

Largest Power Stations:
	•	Loy Yang Power Station (Victoria): Located in Victoria’s Latrobe Valley, Loy Yang consists of two sections: Loy Yang A and Loy Yang B. Combined, they form the largest power station in Australia by capacity.  ￼
	•	Eraring Power Station (New South Wales): Situated in New South Wales, Eraring is one of Australia’s largest individual power stations.  ￼

Dispatchable Unit Identifier (DUID):

In the context of the Australian National Electricity Market (NEM), a Dispatchable Unit Identifier (DUID) is a unique identifier assigned by the Australian Energy Market Operator (AEMO) to each dispatchable unit. A dispatchable unit refers to a specific generating unit within a power station, rather than the entire power station or the owning company. For example, Bayswater Power Station comprises four generating units, each with its own DUID (e.g., ‘BW01’ for Unit 1).  ￼

Therefore, each DUID corresponds to an individual generating unit within a power station, not to the power station as a whole or the company that owns it.

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
from nemosis import static_table, dynamic_data_compiler
import plotly.express as px

In [3]:
raw_data_cache = '/Volumes/T7/NEMO_data'

In [4]:
# Download the latest Generators and Scheduled Loads table. The 
# update_static_file=True argument forces nemosis to download a new copy of 
# file from AEMO even if a copy already exists in the cache.
dispatch_units = static_table(table_name='Generators and Scheduled Loads', 
                              raw_data_location=raw_data_cache,
                              update_static_file=False)


INFO: Retrieving static table Generators and Scheduled Loads


In [7]:
dispatch_units.describe()

,Participant,Station Name,Region,Dispatch Type,Category,Classification,Fuel Source - Primary,Fuel Source - Descriptor,Technology Type - Primary,Technology Type - Descriptor,Aggregation,DUID
count,524,524,524,524,524,524,524,524,524,524,524,524
unique,221,398,5,3,3,6,8,21,4,19,3,524
top,Hydro-Electric Corporation,Adelaide Desalination Plant,NSW1,Generating Unit,Market,Scheduled,Fossil,Solar,Renewable,Wind - Onshore,Y,ADPBA1G
freq,31,7,139,485,515,208,160,118,292,103,391,1


In [8]:
dispatch_units

,Participant,Station Name,Region,Dispatch Type,Category,Classification,Fuel Source - Primary,Fuel Source - Descriptor,Technology Type - Primary,Technology Type - Descriptor,Aggregation,DUID
0,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,ADPBA1G
1,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Load,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,ADPBA1L
2,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Non-Scheduled,Hydro,Water,Renewable,Run of River,Y,ADPMH1
3,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Semi-Scheduled,Solar,Solar,Renewable,Photovoltaic Tracking Flat panel,Y,ADPPV1
4,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Non-Scheduled,Solar,Solar,Renewable,Photovoltaic Flat panel,Y,ADPPV2
...,...,...,...,...,...,...,...,...,...,...,...,...
527,Tailem Bend II Project Company Pty Ltd as trus...,Tailem Bend 2 Hybrid Renewable Power Station,SA1,Bidirectional Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,TB2B1
528,AGL Macquarie Pty Limited,Broken Hill Battery Energy Storage System,NSW1,Bidirectional Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,BHB1
529,AGL SA Generation Pty Limited,Torrens Island BESS,SA1,Bidirectional Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,TIB1
530,Capital Battery Pty Ltd as Trustee for Capital...,Capital Battery,NSW1,Load,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,CAPBES1


How to filter?
1. Remove non-scheduled
2. Filter for generators

In [10]:
# Filter for only generators (exclude loads) and scheduled/semi-scheduled units
dispatched_generators = dispatch_units[
    # Filter for generators
    (dispatch_units['Dispatch Type'] == 'Generating Unit') &
    # Filter for scheduled and semi-scheduled units
    (dispatch_units['Classification'].isin(['Scheduled', 'Semi-Scheduled']))
]

# Show the number of units before and after filtering
print(f"Original number of units: {len(dispatch_units)}")
print(f"Number of dispatched generators: {len(dispatched_generators)}")

# Optional: Show the breakdown of what types we kept
print("\nBreakdown of generator types:")
print(dispatched_generators['Classification'].value_counts())

Original number of units: 524
Number of dispatched generators: 351

Breakdown of generator types:
Classification
Semi-Scheduled    180
Scheduled         171
Name: count, dtype: int64


In [11]:
dispatched_generators.describe()

,Participant,Station Name,Region,Dispatch Type,Category,Classification,Fuel Source - Primary,Fuel Source - Descriptor,Technology Type - Primary,Technology Type - Descriptor,Aggregation,DUID
count,351,351,351,351,351,351,351,351,351,351,351,351
unique,161,260,5,1,2,2,5,13,3,15,2,351
top,Hydro-Electric Corporation,Gladstone Power Station,QLD1,Generating Unit,Market,Semi-Scheduled,Fossil,Solar,Renewable,Wind - Onshore,Y,ADPBA1G
freq,25,6,90,351,350,180,127,103,221,76,313,1
